In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta

## データの読み込み

In [2]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [3]:
display(df_train.head())
display(df_test.head())

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature
0,2013-11-18,90,月,0,厚切りイカフライ,NaN,NaN,NaN,NaN,快晴,--,19.8
1,2013-11-19,101,火,1,手作りヒレカツ,NaN,NaN,NaN,NaN,快晴,--,17.0
2,2013-11-20,118,水,0,白身魚唐揚げ野菜あん,NaN,NaN,NaN,NaN,快晴,--,15.5
3,2013-11-21,120,木,1,若鶏ピリ辛焼,NaN,NaN,NaN,NaN,快晴,--,15.2
4,2013-11-22,130,金,1,ビッグメンチカツ,NaN,NaN,NaN,NaN,快晴,--,16.1


,datetime,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature
0,2014-10-1,水,1,メンチカツ,420.0,NaN,NaN,NaN,雨,0,20.2
1,2014-10-2,木,0,バーベキューチキン,415.0,NaN,NaN,NaN,曇,--,23.9
2,2014-10-3,金,0,豚肉のマスタード焼き,405.0,NaN,NaN,NaN,晴れ,--,28.7
3,2014-10-6,月,1,麻婆春雨,400.0,NaN,NaN,NaN,雨,0.5,21.5
4,2014-10-7,火,0,厚揚げ肉みそ炒め,430.0,NaN,NaN,NaN,晴れ,--,22.1


## trainとtestは結合して処理する
- メニューやイベントにおいてどちらかにないというのにフラグがたてられないため

In [4]:
df_train.loc[:, "data_flg"] = 1
df_test.loc[:, "data_flg"] = 0
df_test.loc[:, "y"] = np.nan
df_test = df_test[df_train.columns]

In [5]:
dataset_df = pd.concat([df_train, df_test])

## 時間の処理をしておく

In [6]:
dataset_df["datetime"] = pd.to_datetime(dataset_df["datetime"])

## カテゴリ化する

In [7]:
def make_dummies(df_c):
    source = df_c.copy()
    dummy_df = pd.DataFrame([])
    dummy_col = ["week", "remarks", "event"]
    for col in dummy_col:
        dummy = pd.get_dummies(df_c[col])
        dummy_df = pd.concat([dummy_df, dummy], axis=1)
    source = source.drop(dummy_col, axis = 1)
    
    return pd.concat([source, dummy_df], axis=1)

In [8]:
dataset_df = make_dummies(dataset_df)

## 天気を数値へ変換する
|天気コード|天気|
|-------|----|
|快晴|1|
|晴れ|2|
|薄曇|3|
|曇  |4|
|雨  |5|
|雪  |6|
|雷電 |7|

In [9]:
weather_dict = {"快晴":1, "晴れ":2, "薄曇":3, "曇":4, "雨":5, "雪":6, "雷電":7}

In [10]:
dataset_df["weather"] = dataset_df["weather"].map(lambda x: weather_dict[x])

## 降水量の置換

In [11]:
dataset_df["precipitation"] = dataset_df["precipitation"].map(lambda x: 0 if x=="--" else x)

In [12]:
dataset_df.to_csv("dataset_df.csv", index=False)

## 給料日の経過日数

In [13]:
pay_df = pd.read_csv("給料日日付データ.csv")

In [14]:
pay_df["datetime"] = pd.to_datetime(pay_df["datetime"])
pay_df["経過終了日"] = pay_df["datetime"]
pay_df = pay_df.set_index("datetime")
pay_day = pay_df[pay_df["給料日"] == 1]
pay_day["経過開始日"] = pay_day["経過終了日"].shift()
pay_day["経過終了日"] = pay_day["経過終了日"] - timedelta(days=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
keika_df = pd.DataFrame([])
for d in pay_day[["経過開始日", "経過終了日"]].values:
    temp = pay_df[d[0]: d[1]].reset_index(drop=False).reset_index(drop=False)
    temp = temp[["datetime","index"]]
    keika_df = pd.concat([keika_df, temp])
keika_df = keika_df.rename(columns={"index": "給料日経過日数"})
keika_df["給料日経過日数"] = keika_df.loc[:, "給料日経過日数"] + 1

In [16]:
keika_df.head()

,datetime,給料日経過日数
0,2013-11-08,1
1,2013-11-09,2
2,2013-11-10,3
3,2013-11-11,4
4,2013-11-12,5


In [17]:
dataset_df = pd.merge(dataset_df, keika_df, how="left", on="datetime")
dataset_df = dataset_df.drop("payday", axis=1)

## kcalの欠損を埋める

In [18]:
menu_df = pd.read_csv("メニュー分類.csv")
menu_columns = list(menu_df.columns)
menu_ = list(menu_df.columns)
menu_.remove("name")
menu_columns.append("kcal")

In [19]:
#おいちゃん特性のメニュー分類の結合
dataset_df = pd.merge(dataset_df, menu_df, how="left", on="name")

In [20]:
# 欠損と欠損じゃないのを分ける
no_missing_tf = dataset_df["kcal"] == dataset_df["kcal"]
no_missing_data = dataset_df[no_missing_tf]
missing_data = dataset_df[~no_missing_tf]
print ("欠損メニュー数：",missing_data["kcal"].isnull().sum())

欠損メニュー数： 45


#### 欠損を除外してkcalの平均を算出
#### 重複メニューにkcalが入っていた場合はそれを使う

In [21]:
equal_name_kcal_dict = pd.DataFrame(no_missing_data.groupby("name").mean()["kcal"]).to_dict()

In [22]:
for i in equal_name_kcal_dict["kcal"].keys():
    missing_data.loc[missing_data["name"] == i, "kcal"] = equal_name_kcal_dict["kcal"][i]

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [23]:
print ("欠損メニュー数：",missing_data["kcal"].isnull().sum())

欠損メニュー数： 33


#### その他はおいちゃん特性のメニュー分類で埋める

In [24]:
for i in menu_:
    equal_menu_mean = no_missing_data[no_missing_data[i] == 1]["kcal"].mean()
    print ("メニュー：{0} {1}".format(i, equal_menu_mean))
    missing_data.loc[(missing_data[i] == 1) & (missing_data["kcal"].isnull()), "kcal"] = equal_menu_mean

メニュー：カツ 411.10714285714283
メニュー：カレー 410.0
メニュー：ハンバーグ 408.8333333333333
メニュー：中華 405.0408163265306
メニュー：フライ 422.0
メニュー：チキン 408.1666666666667
メニュー：その他 406.0813953488372


In [25]:
print ("欠損メニュー数：",missing_data["kcal"].isnull().sum())

欠損メニュー数： 0


In [26]:
fin_dataset = pd.concat([no_missing_data, missing_data])

In [27]:
fin_dataset = fin_dataset.sort_values("datetime")

In [28]:
fin_dataset.to_csv("dataset_df.csv", index=False)

## 後処理

In [30]:
fin_train_data = fin_dataset[fin_dataset["data_flg"] == 1].reset_index(drop=True)
fin_train_data = fin_train_data.drop(["name", "data_flg"], axis=1)
fin_test_data = fin_dataset[fin_dataset["data_flg"] == 0].reset_index(drop=True)
fin_test_data = fin_test_data.drop(["name", "data_flg", "y"], axis=1)

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
train_data, val_data = train_test_split(fin_train_data, test_size=0.2, shuffle=False)

In [33]:
print (train_data.shape)
print (val_data.shape)
print (fin_test_data.shape)

(165, 29)
(42, 29)
(40, 28)


In [35]:
train_data.to_csv("output_train_data.csv", index=False)
val_data.to_csv("output_val_data.csv", index=False)
fin_test_data.to_csv("output_test_data", index=False)